In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries which we will use through out this notebook

In [ ]:
# PyTorch Imports
import torch
import torch.nn as nn                               # to access build-in functions to build the NN
import torch.nn.functional as F                     # to access activation functions
import torch.optim as optim                         # to build out optimizer
from torch.autograd import Variable
from torch.utils.data import DataLoader





import matplotlib.pyplot as plt
from PIL import Image
from keras.utils import to_categorical
%matplotlib inline

# Loading Dataset

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv') 

* Viewing dataset

In [ ]:
train.head()

In [ ]:
train_set = train.iloc[:,1:]
label = train["label"]

# Normalizing

In [ ]:
# normalizing

norm_train_set = train_set / 255
norm_test_set = test / 255

# Preparing our data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validate, y_train, y_validate = train_test_split(norm_train_set, label, test_size = 0.1)

In [ ]:
X_train = torch.from_numpy(X_train.values.reshape(-1,1,28,28))
X_validate = torch.from_numpy(X_validate.values.reshape(-1,1,28,28))
testing_set = torch.from_numpy(norm_test_set.values.reshape(-1,1,28,28))
y_train = torch.from_numpy(y_train.values)
y_validate = torch.from_numpy(y_validate.values)

In [ ]:
training_set = torch.utils.data.TensorDataset(X_train.float(), y_train)
validating_set = torch.utils.data.TensorDataset(X_validate.float(), y_validate)
testing_set = torch.utils.data.TensorDataset(testing_set.float())

In [ ]:
train_loader = DataLoader(training_set, shuffle=True, batch_size = 88)
validate_loader = DataLoader(validating_set, shuffle=False, batch_size = 88)
test_set = DataLoader(testing_set, shuffle=False, batch_size = 88)

# Visualizing Some data

In [ ]:
# Check a sample of the images

X = train.iloc[:,1:].values.reshape(-1,1,28,28)
y = train["label"].values

plt.figure(figsize = (16, 3))


ind = 0

for i in range(16):
    if i >= 16:
        break
    plt.subplot(2, 8, i+1)
    plt.title(y[i])
    plt.imshow(X[i].reshape(28,28))
        

# Building CNN model using PyTorch

In [ ]:
class CNN_DigitClassifier(nn.Module):
    def __init__(self):
        super(CNN_DigitClassifier, self).__init__()
        
        self.features = nn.Sequential(
            
                        nn.Conv2d(1, 32, 5),
                        nn.ReLU(inplace=True),
                        nn.Conv2d(32, 32, 5),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(2,2), 
                        nn.Dropout(0.25),
            
            
                        nn.Conv2d(32, 64, 3),
                        nn.ReLU(inplace=True),
                        nn.Conv2d(64, 64, 3),
                        nn.ReLU(inplace=True),
                        nn.MaxPool2d(2), 
                        nn.Dropout(0.25)) 
        
        self.classification = nn.Sequential(
                                nn.Linear(576, 256),
                                nn.Dropout(0.25),
                                nn.Linear(256, 10)) ## 10 possible prediction
        
    def forward(self, images):
        
        images = self.features(images)
        
        images = images.view(images.shape[0], -1)
        
        output = self.classification(images)
        
        return output

In [ ]:

model = CNN_DigitClassifier()


optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9)

criterion = nn.CrossEntropyLoss()

lr_reduction = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0.00001)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

# Training and Evaluating

In [ ]:
count = 0
losses = []
iteration_list = []
training_accuracy = []
validation_accuracy = []
training_loss = []
validation_loss = []

In [ ]:
def train(epoch):
    
    global count
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        if (batch_idx + 1)% 100 == 0:
            losses.append(loss.item())
            iteration_list.append(count)
            count += 1

In [ ]:
def evaluate(data_loader, validate=False):
    model.eval()
    loss = 0
    correct = 0
    
    for data, target in data_loader:
        data, target = Variable(data), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += F.cross_entropy(output, target, size_average=False).item()

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
    
    accuracy = 100. * correct / len(data_loader.dataset)
    
    if not validate:
        lr_reduction.step(loss)
        training_accuracy.append(accuracy)
        training_loss.append(loss)
    else:
        validation_accuracy.append(accuracy)
        validation_loss.append(loss)

In [ ]:
n_epochs = 50


for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)
    evaluate(validate_loader, True)
    
plt.plot(iteration_list,losses)
plt.xlabel("Number of iteration")
plt.ylabel("Loss")
plt.title("Training Loss vs Number of iteration")
plt.show()

epoch_list = [i for i in range(n_epochs)]

plt.plot(epoch_list, training_loss)
plt.plot(epoch_list, validation_loss)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss vs Epoch")
plt.show()

plt.plot(epoch_list, training_accuracy)
plt.plot(epoch_list, validation_accuracy)
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Epoch")
plt.show()

# Prediction

In [ ]:
def prediction(data_loader):
    model.eval()
    test_pred = torch.LongTensor()
    
    for batch_idx, data in enumerate(data_loader):
        data = Variable(data[0])
        if torch.cuda.is_available():
            data = data.cuda()
            
        output = model(data)
        
        pred = output.cpu().data.max(1, keepdim=True)[1]
        test_pred = torch.cat((test_pred, pred), dim=0)
        
    return test_pred



In [ ]:
test_prediction = prediction(test_set)

# Submission

In [ ]:

y_test_pred = test_prediction.numpy().ravel()
y_test_pred = pd.Series(y_test_pred, name="Label")

submission = pd.concat([pd.Series(range(1,28001), name = "ImageId"), y_test_pred], axis = 1)



In [ ]:
submission.head()

In [ ]:
submission.to_csv("CNN_model_TPU_submission.csv", index = False)